In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, log_loss
import operator
import json
from IPython import display
import os
import warnings

np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

Task: To predict whether the user likes the mobile phone or not. <br>
Assumption: If the average rating of mobile >= threshold, then the user likes it, otherwise not.

<b>Missing values:</b><br>
'Also Known As'(459),'Applications'(421),'Audio Features'(437),'Bezel-less display'(266),'Browser'(449),'Build Material'(338),'Co-Processor'(451),'Display Colour'(457),'Mobile High-Definition Link(MHL)'(472),'Music'(447)
'Email','Fingerprint Sensor Position'(174),'Games'(446),'HDMI'(454),'Heart Rate Monitor'(467),'IRIS Scanner'(467),
'Optical Image Stabilisation'(219),'Other Facilities'(444),'Phone Book'(444),'Physical Aperture'(87),'Quick Charging'(122),'Ring Tone'(444),'Ruggedness'(430),SAR Value(315),'SIM 3'(472),'SMS'(470)', 'Screen Protection'(229),'Screen to Body Ratio (claimed by the brand)'(428),'Sensor'(242),'Software Based Aperture'(473),
'Special Features'(459),'Standby time'(334),'Stylus'(473),'TalkTime'(259), 'USB Type-C'(374),'Video Player'(456),
'Video Recording Features'(458),'Waterproof'(398),'Wireless Charging','USB OTG Support'(159), 'Video ,'Recording'(113),'Java'(471),'Browser'(448)

<b>Very low variance:</b><br>
'Architecture'(most entries are 64-bit),'Audio Jack','GPS','Loudspeaker','Network','Network Support','Other Sensors'(28),'SIM Size', 'VoLTE'


<b>Multivalued:</b><br>
'Colours','Custom UI','Model'(1),'Other Sensors','Launch Date'

<b>Not important:</b><br>
'Bluetooth', 'Settings'(75),'Wi-Fi','Wi-Fi Features'

<b>Doubtful:</b><br>
'Aspect Ratio','Autofocus','Brand','Camera Features','Fingerprint Sensor'(very few entries are missing),
'Fingerprint Sensor Position', 'Graphics'(multivalued),'Image resolution'(multivalued),'SIM Size','Sim Slot(s)', 'User Available Storage', 'SIM 1', 'SIM 2','Shooting Modes', 'Touch Screen'(24), 'USB Connectivity'
    
<b>To check:</b><br>
'Display Type','Expandable Memory','FM Radio'

<b>High Correlation with other features</b><br>
'SIM Slot(s)' high correlation with SIM1
'Weight' has high high correlation with capacity , screen-to-body ratio
'Height' - screen size is also there
    
<b>Given a mobile, we can't directly get these features</b><br>
'Rating Count', 'Review Count'

<b>Keeping:</b><br>
'Capacity','Flash'(17),'Height'(22),'Internal Memory'(20, require cleaning),'Operating System'(25, require cleaning), 'Pixel Density'(1, clean it),'Processor'(22, clean it), 'RAM'(17, clean), 'Rating','Resolution'(cleaning), 'Screen Resolution','Screen Size', 'Thickness'(22), 'Type','User Replaceable','Weight'(cleaning),'Sim Size'(), 'Other Sensors'(28), 'Screen to Body Ratio (calculated)','Width',


In [5]:
# read data from file
train = pd.read_csv("./input/train.csv") 
test = pd.read_csv("./input/test.csv")

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 355
Number of features in train: 99
Number of data points in test: 119
Number of features in test: 98


In [6]:
def data_clean(data):
    
    # Let's first remove all missing value features
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

# Removing features

In [7]:
train = data_clean(train)
test = data_clean(test)

removing all those data points in which more than 15 features are missing 

In [8]:
train = train[(train.isnull().sum(axis=1) <= 15)]
# You shouldn't remove data points from test set
#test = test[(test.isnull().sum(axis=1) <= 15)]

In [9]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


# Filling Missing values

In [10]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass


In [11]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [12]:
train = data_clean_2(train)
test = data_clean_2(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


Not very important feature

In [13]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [14]:
train = data_clean_3(train)
test = data_clean_3(test)

# check the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [15]:
# one hot encoding

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

(460, 17)
Index(['PhoneId', 'Sim1', 'Capacity', 'Weight', 'RAM', 'Num_cores',
       'Resolution', 'Brand', 'SIM 2', 'Processor_frequency', 'Height',
       'Internal Memory', 'os_name', 'Screen Size', 'SIM Slot(s)',
       'Screen to Body Ratio (calculated)', 'Pixel Density'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Capacity', 'Weight', 'RAM', 'Resolution',
       'Processor_frequency', 'Height', 'Internal Memory', 'Screen Size',
       'Screen to Body Ratio (calculated)', 'Pixel Density', 'Sim1_2G',
       'Sim1_3G', 'Sim1_4G', 'Num_cores_312', 'Num_cores_Deca',
       'Num_cores_Dual', 'Num_cores_Hexa', 'Num_cores_Octa', 'Num_cores_Other',
       'Num_cores_Quad', 'Num_cores_Tru-Octa', 'Brand_10.or', 'Brand_Apple',
       'Brand_Asus', 'Brand_Billion', 'Brand_Blackberry', 'Brand_Comio',
       'Brand_Coolpad', 'Brand_Do', 'Brand_Gionee', 'Brand_Google',
       'Brand_HTC', 'Brand_Honor', 'Brand_Huawei', 'Brand_InFocus',
       'Brand_Infinix', 'Brand_Intex', 'Brand_Itel',

In [16]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [17]:
# check the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# check the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [19]:
train_new.head()

,PhoneId,Capacity,Weight,RAM,Resolution,Processor_frequency,Height,Internal Memory,Screen Size,Screen to Body Ratio (calculated),...,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",Rating
0,0,4000,182,4,20,1.8,157.9,64,6.26,80.68,...,False,False,False,False,False,False,False,True,False,4.5
1,1,4230,168,3,8,1.8,156.2,32,6.20,80.85,...,False,False,False,False,False,False,False,True,False,4.5
2,2,3500,168,3,25,2.1,157.0,32,6.30,83.68,...,False,False,False,False,False,False,False,True,False,4.4
3,4,3300,169,4,24,2.2,159.8,64,6.00,74.78,...,False,False,False,False,False,False,True,False,False,4.3
4,5,3750,175,4,16,2.2,160.4,64,6.50,84.23,...,False,False,False,False,False,False,False,True,False,4.4


In [20]:
test_new.head()

,PhoneId,Capacity,Weight,RAM,Resolution,Processor_frequency,Height,Internal Memory,Screen Size,Screen to Body Ratio (calculated),...,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM"
0,3,4230,168,2,5,1.8,156.2,16,6.20,80.85,...,False,False,False,False,False,False,False,False,True,False
1,11,5000,205,4,12,1.8,156.0,64,6.20,81.60,...,False,False,False,False,False,False,False,False,True,False
2,13,3500,169,6,25,2.0,156.7,64,6.30,83.84,...,False,False,False,False,False,False,False,False,True,False
3,16,3500,169,4,16,2.0,156.7,64,6.30,83.84,...,False,False,False,False,False,False,False,False,True,False
4,19,4000,181,4,20,1.8,158.6,64,5.99,77.43,...,False,False,False,False,False,False,False,False,True,False


## Dummy Solution

In [21]:
submission = pd.DataFrame({'PhoneId':test_new['PhoneId'], 'Class':[0]*test_new.shape[0]})
submission = submission[['PhoneId', 'Class']]
submission.head()

,PhoneId,Class
0,3,0
1,11,0
2,13,0
3,16,0
4,19,0


In [22]:
submission.to_csv("submission.csv", index=False)

In [24]:
train_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 88 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   PhoneId                                    341 non-null    int64  
 1   Capacity                                   341 non-null    int64  
 2   Weight                                     341 non-null    int64  
 3   RAM                                        341 non-null    int64  
 4   Resolution                                 341 non-null    int64  
 5   Processor_frequency                        341 non-null    float64
 6   Height                                     341 non-null    float64
 7   Internal Memory                            341 non-null    int64  
 8   Screen Size                                341 non-null    float64
 9   Screen to Body Ratio (calculated)          341 non-null    float64
 10  Pixel Density             

In [32]:
(train_new['Capacity']>3000).sum()

np.int64(182)

In [35]:
train_new.describe()

,PhoneId,Capacity,Weight,RAM,Resolution,Processor_frequency,Height,Internal Memory,Screen Size,Screen to Body Ratio (calculated),Pixel Density,Rating
count,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000,341.000000
mean,232.970674,3191.307918,161.343109,11.120235,9.592375,1.792551,149.946394,46.369501,5.462317,72.353343,344.211144,4.072434
std,137.231746,932.533527,22.305556,61.320855,5.953827,0.451598,9.192305,46.435453,0.742529,9.357839,88.161129,0.377066
min,0.000000,1020.000000,58.000000,1.000000,2.000000,1.000000,115.600000,0.000000,2.400000,23.600000,143.000000,2.500000
25%,110.000000,2900.000000,150.000000,2.000000,5.000000,1.400000,146.200000,16.000000,5.200000,68.800000,282.000000,3.900000
50%,229.000000,3055.000000,161.000000,3.000000,8.000000,1.800000,151.300000,32.000000,5.500000,72.500000,326.000000,4.100000
75%,354.000000,3750.000000,172.000000,4.000000,13.000000,2.200000,156.200000,64.000000,6.000000,77.480000,403.000000,4.300000
max,472.000000,13000.000000,330.000000,512.000000,25.000000,2.800000,174.100000,512.000000,6.590000,87.660000,576.000000,5.000000


In [48]:
# Get the class column
train_new['Class'] = train_new['Rating']>=train_new['Rating'].quantile()
train_new.drop("Rating", axis=1,inplace=True)

In [49]:
train_new.head()

,PhoneId,Capacity,Weight,RAM,Resolution,Processor_frequency,Height,Internal Memory,Screen Size,Screen to Body Ratio (calculated),...,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",Class
0,0,4000,182,4,20,1.8,157.9,64,6.26,80.68,...,False,False,False,False,False,False,False,True,False,True
1,1,4230,168,3,8,1.8,156.2,32,6.20,80.85,...,False,False,False,False,False,False,False,True,False,True
2,2,3500,168,3,25,2.1,157.0,32,6.30,83.68,...,False,False,False,False,False,False,False,True,False,True
3,4,3300,169,4,24,2.2,159.8,64,6.00,74.78,...,False,False,False,False,False,False,True,False,False,True
4,5,3750,175,4,16,2.2,160.4,64,6.50,84.23,...,False,False,False,False,False,False,False,True,False,True


In [57]:
from pandas.api.types import is_bool_dtype, is_numeric_dtype

def preprocess_data(df):
    """Preprocessing function."""
    df_new = df.copy()

    # Drop the PhoneID column (assuming 'PhoneID' is the correct name)
    df_new.drop("PhoneId", axis=1, inplace=True)

    for column in df_new.columns:
        if is_bool_dtype(df_new[column]):
            df_new[column] = df_new[column].astype(int)
        elif is_numeric_dtype(df_new[column]):
            # Quantile-based conversion
            threshold = df_new[column].quantile()
            df_new[column] = (df_new[column] >= threshold).astype(int)
        else:
            pass

    return df_new


In [58]:
train_new_1 = preprocess_data(train_new)
train_new_1.head()

,Capacity,Weight,RAM,Resolution,Processor_frequency,Height,Internal Memory,Screen Size,Screen to Body Ratio (calculated),Pixel Density,...,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",Class
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,1
1,1,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,1,0,1
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,1
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,1
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,0,1


In [61]:
test_new.head()

,PhoneId,Capacity,Weight,RAM,Resolution,Processor_frequency,Height,Internal Memory,Screen Size,Screen to Body Ratio (calculated),...,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM"
0,3,4230,168,2,5,1.8,156.2,16,6.20,80.85,...,False,False,False,False,False,False,False,False,True,False
1,11,5000,205,4,12,1.8,156.0,64,6.20,81.60,...,False,False,False,False,False,False,False,False,True,False
2,13,3500,169,6,25,2.0,156.7,64,6.30,83.84,...,False,False,False,False,False,False,False,False,True,False
3,16,3500,169,4,16,2.0,156.7,64,6.30,83.84,...,False,False,False,False,False,False,False,False,True,False
4,19,4000,181,4,20,1.8,158.6,64,5.99,77.43,...,False,False,False,False,False,False,False,False,True,False


In [62]:
test_new_1 = preprocess_data(test_new)
test_new_1.head()

,Capacity,Weight,RAM,Resolution,Processor_frequency,Height,Internal Memory,Screen Size,Screen to Body Ratio (calculated),Pixel Density,...,os_name_Blackberry,os_name_KAI,os_name_Nokia,os_name_Other,os_name_Tizen,os_name_iOS,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM"
0,1,1,0,0,1,1,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,1,0


In [63]:
X_train = train_new_1.drop("Class", axis=1).values
y_train = train_new_1.Class.values
x_test = test_new_1.values

In [64]:
from sklearn.metrics import accuracy_score

In [66]:
class MP_Neuron:
    def __init__(self):
        self.b=None
        
    def model(self,x):
        return (sum(x)>=self.b)
    
    def predict(self,X):
        Y=[]
        for x in X:
            y=self.model(x)
            Y.append(y)
        return np.array(Y)
    
    def fit(self,X,Y):
        accuracy={}
        for b in range(X.shape[1]+1):
            self.b=b
            Y_pred=self.predict(X)
            accuracy[b]=accuracy_score(Y_pred,Y)
        best_b=max(accuracy,key=accuracy.get)
        self.b=best_b
        print("The optimal value of threshold is=",self.b," with an accuracy=",accuracy[self.b])  
        return accuracy


In [68]:
mp_neuron=MP_Neuron()
mp_neuron.fit(X_train,y_train)

The optimal value of threshold is= 9  with an accuracy= 0.6480938416422287


{0: 0.5865102639296188,
 1: 0.5865102639296188,
 2: 0.5865102639296188,
 3: 0.5865102639296188,
 4: 0.5865102639296188,
 5: 0.5865102639296188,
 6: 0.5865102639296188,
 7: 0.6129032258064516,
 8: 0.6451612903225806,
 9: 0.6480938416422287,
 10: 0.6304985337243402,
 11: 0.6451612903225806,
 12: 0.6363636363636364,
 13: 0.6304985337243402,
 14: 0.5689149560117303,
 15: 0.5542521994134897,
 16: 0.5131964809384164,
 17: 0.41348973607038125,
 18: 0.41348973607038125,
 19: 0.41348973607038125,
 20: 0.41348973607038125,
 21: 0.41348973607038125,
 22: 0.41348973607038125,
 23: 0.41348973607038125,
 24: 0.41348973607038125,
 25: 0.41348973607038125,
 26: 0.41348973607038125,
 27: 0.41348973607038125,
 28: 0.41348973607038125,
 29: 0.41348973607038125,
 30: 0.41348973607038125,
 31: 0.41348973607038125,
 32: 0.41348973607038125,
 33: 0.41348973607038125,
 34: 0.41348973607038125,
 35: 0.41348973607038125,
 36: 0.41348973607038125,
 37: 0.41348973607038125,
 38: 0.41348973607038125,
 39: 0.413489

In [69]:
y_pred = mp_neuron.predict(x_test)

In [72]:
submission = pd.DataFrame()
submission['PhoneId'] = test_new['PhoneId']
submission['Class'] = y_pred.astype(int)
submission.head()
#submission.to_csv("submission.csv", index=False)

,PhoneId,Class
0,3,1
1,11,1
2,13,1
3,16,1
4,19,1


In [73]:
submission.to_csv("submission.csv", index=False)

In [74]:
# Score: 0.75630